### Verificación del entorno de Python

Antes de empezar con el análisis, verificamos la versión de Python
y el entorno activo para asegurarnos de que el notebook está
usando el intérprete correcto.


In [1]:
import sys
print(sys.version)
print(sys.executable)


3.13.9 | packaged by conda-forge | (main, Oct 22 2025, 23:12:41) [MSC v.1944 64 bit (AMD64)]
c:\Users\Usuario\miniconda3\envs\mienv3\python.exe


### Verificación de Pandas

Comprobamos que la librería Pandas está instalada y la versión utilizada,
ya que será la principal herramienta para manipulación y análisis de datos.


In [2]:
import pandas as pd
print(pd.__version__)


2.3.3


### Carga del dataset de canciones de Spotify

Cargamos el CSV que contiene información de 2000 canciones
con sus características musicales, popularidad, duración, género, etc.
Luego mostramos las primeras filas para tener una idea general
de las columnas y el formato de los datos.


In [3]:
df = pd.read_csv("../data/raw/songs_normalize.csv")
df.head()


,artist,song,duration_ms,explicit,year,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,genre
0,Britney Spears,Oops!...I Did It Again,211160,False,2000,77,0.751,0.834,1,-5.444,0,0.0437,0.3000,0.000018,0.3550,0.894,95.053,pop
1,blink-182,All The Small Things,167066,False,1999,79,0.434,0.897,0,-4.918,1,0.0488,0.0103,0.000000,0.6120,0.684,148.726,"rock, pop"
2,Faith Hill,Breathe,250546,False,1999,66,0.529,0.496,7,-9.007,1,0.0290,0.1730,0.000000,0.2510,0.278,136.859,"pop, country"
3,Bon Jovi,It's My Life,224493,False,2000,78,0.551,0.913,0,-4.063,0,0.0466,0.0263,0.000013,0.3470,0.544,119.992,"rock, metal"
4,*NSYNC,Bye Bye Bye,200560,False,2000,65,0.614,0.928,8,-4.806,0,0.0516,0.0408,0.001040,0.0845,0.879,172.656,pop


### Observaciones iniciales

- El dataset tiene 18 columnas que incluyen información del artista,
  nombre de la canción, características musicales y género.
- Las primeras filas muestran que:
  - `explicit` es booleano,
  - `year` y `popularity` son enteros,
  - `danceability`, `energy` y otras medidas son floats entre 0 y 1.
- La columna `genre` a veces contiene **múltiples géneros separados por comas**.
- Esto nos indica que necesitaremos explorar y limpiar ciertos campos
  antes de hacer análisis más profundos.


### Dimensiones del dataset

Verificamos cuántas filas y columnas tiene el dataset
para entender su tamaño y planificar el análisis.


In [4]:
df.shape

(2000, 18)

### Observaciones sobre dimensiones

- El dataset contiene **2000 canciones** (filas) y **18 columnas**.  
- Esto confirma que tenemos un tamaño manejable para análisis exploratorio y visualizaciones.  
- Cada fila representa una canción única y cada columna una característica o métrica relacionada con la canción.
